In [1]:
import tensorflow as tf
import numpy as np
import time
import math
import os
import cv2
import sys

In [2]:
'''
初始化权重函数
'''
def variable_with_weight_loss(shape, std, w1):
    var = tf.Variable(tf.truncated_normal(shape,stddev=std),dtype=tf.float32)
    
    if w1 is not None :
        weight_loss = tf.multiply(tf.nn.l2_loss(var),w1,name="weight_loss")
        tf.add_to_collection("losses",weight_loss)
    return var

'''
损失函数
'''
def loss_func(logits,labels):
    labels = tf.cast(labels,tf.int32)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                           labels=labels,name="cross_entropy_per_example")
    cross_entropy_mean = tf.reduce_mean(tf.reduce_sum(cross_entropy))
    tf.add_to_collection("losses",cross_entropy_mean)
    return tf.add_n(tf.get_collection("losses"),name="total_loss")



    
def forward(image_holder,train):
    
    #设计第一层卷积
    weight1 = variable_with_weight_loss(shape=[5,5,3,64],std=5e-2,w1=0)
    kernel1 = tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding="SAME")
    bais1 = tf.Variable(tf.constant(0.0,dtype=tf.float32,shape=[64]))
    conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bais1))
    pool1 = tf.nn.max_pool(conv1,[1,3,3,1],[1,2,2,1],padding="SAME")
    norm1 = tf.nn.lrn(pool1,4,bias=1.0,alpha=0.001 / 9,beta=0.75)

    #设计第二层卷积
    weight2 = variable_with_weight_loss(shape=[5,5,64,64],std=5e-2,w1=0)
    kernel2 = tf.nn.conv2d(norm1,weight2,[1,1,1,1],padding="SAME")
    bais2 = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(kernel2,bais2))
    norm2 = tf.nn.lrn(conv2,4,bias=1.0,alpha=0.01 / 9,beta=0.75)
    pool2 = tf.nn.max_pool(norm2,[1,3,3,1],[1,2,2,1],padding="SAME")

    #第一层全连接层
    # 把pool2从三维张量变为二维张量
    pool_shape = pool2.get_shape().as_list()         # 得到pool2 输出矩阵的维度存入list中
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]      # 提取特征的长度宽度深度 相乘得所有特征点的个数
    reshape = tf.reshape(pool2, [pool_shape[0], nodes])       # pool_shape[0]是一个batch的值
    #reshape = tf.reshape(pool2,[batch_size,-1])
    #dim = reshape.get_shape()[1].value
    weight3 = variable_with_weight_loss([nodes,384],std=0.04,w1=0.004)
    bais3 = tf.Variable(tf.constant(0.1,shape=[384],dtype=tf.float32))
    local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bais3)
    #if train:
        # 如果是训练阶段，在这层使用0.5的dropout
        #local3 = tf.nn.dropout(local3, 0.2)
    #第二层全连接层
    weight4 = variable_with_weight_loss([384,192],std=0.04,w1=0.004)
    bais4 = tf.Variable(tf.constant(0.1,shape=[192],dtype=tf.float32))
    local4 = tf.nn.relu(tf.matmul(local3,weight4)+bais4)

    #最后一层
    weight5 = variable_with_weight_loss([192,10],std=1/192.0,w1=0)
    bais5 = tf.Variable(tf.constant(0.0,shape=[10],dtype=tf.float32))
    
    logits = tf.add(tf.matmul(local4,weight5),bais5)
    return logits

In [ ]:

# 定义TFRecord解析函数
def parser(record):
    print(record)
    features = tf.parse_single_example(record,
                                       features={
                                           'image' : tf.FixedLenFeature([], tf.string),
                                           'label': tf.FixedLenFeature([], tf.int64),
                                       })
    # 获取图片数据
    images = tf.decode_raw(features['image'], tf.uint8)
    
    # tf.train.shuffle_batch必须确定shape
    images = tf.reshape(images, [24*24*3])
    
    # 图片预处理
    images = tf.cast(images, tf.float32) / 255.0
    images = tf.subtract(images, 0.5)
    images = tf.multiply(images, 2.0)
    # 获取label
    labels = tf.cast(features['label'], tf.int32)
    
    return images, labels


In [ ]:
if __name__ == '__main__':

    #设置最大迭代次数
    max_steps = 60001
    #设置每次训练的数据大小
    batch_size = 100
    num_classes = 10
    epochs = 16
    buffer_size = 1000
    WIDTH = 24 
    HEIGHT = 24
    CHANNELS = 3
    BATCH_SIZE = 100
    LEARNING_RATE_DECAY = 0.96
    LEARNING_RATE_BASE = 0.001
    # tfrecord文件存放路径
    TFRECORD_FILE_TRAIN = ['captcha/train.tfrecords']
    TFRECORD_FILE_TEST = ['captcha/test.tfrecords']
    test_dir = 'D:/Develop/project/python/tensorflow/ten/param/captcha/images/test'
    train_dir = 'D:/Develop/project/python/tensorflow/ten/param/captcha/images/train'
    #获取数据增强后的训练集数据
    #images_train,labels_train = load_files(train_dir,num_classes,24*24*3)
    # 全局步数
    global_step = tf.Variable(0, trainable=False) 
    
    # 创建训练集和测试集对象 使用 map() 进行解析
    dataset_train =  tf.data.TFRecordDataset(TFRECORD_FILE_TRAIN)
    dataset_test =  tf.data.TFRecordDataset(TFRECORD_FILE_TEST)
    dataset_train = dataset_train.map(parser)
    dataset_test = dataset_test.map(parser)
    dataset_train = dataset_train.repeat().shuffle(buffer_size).batch(batch_size)
    # 创建迭代器
    iterator_train = dataset_train.make_one_shot_iterator()
    iterator_test = dataset_test.make_one_shot_iterator()
    # 创建遍历器
    train_next_batch  = iterator_train.get_next()
    test_next_batch = iterator_test.get_next()
        
    
    #定义模型的输入和输出数据
    image_holder = tf.placeholder(dtype=tf.float32,shape=[batch_size,24*24*3])
    label_holder = tf.placeholder(dtype=tf.int32,shape=[batch_size])
    image_holder_reshape = tf.reshape(image_holder, [-1, WIDTH, HEIGHT, CHANNELS])
    logits = forward(image_holder_reshape,True)
    # 学习率
    lr = tf.Variable(1e-3, dtype=tf.float32)

    # 设置指数下降学习率
    lr_decay = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True) 
    #获取损失函数
    loss = loss_func(logits,label_holder)

    #设置优化算法使得成本最小
    train_step = tf.train.AdamOptimizer(lr).minimize(loss,global_step=global_step)
    #获取最高类的分类准确率，取top1作为衡量标准
    #top_k_op = tf.nn.in_top_k(logits,label_holder,1)
    # 计算准确率
    # 把标签转成one_hot的形式
    one_hot_labels = tf.one_hot(indices=tf.cast(label_holder, tf.int32), depth=num_classes)
    correct_prediction = tf.equal(tf.argmax(one_hot_labels,1),tf.argmax(logits,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#     #计算批次                           
#     batches_count_train = int(math.ceil(images_train.shape[0] / batch_size))
#     remainder_train = images_train.shape[0] % batch_size 
#     batches_count_test = int(math.ceil(images_test.shape[0] / batch_size))
#     remainder_test = images_test.shape[0] % batch_size
    

    #创建会话
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # 用于保存模型
    saver = tf.train.Saver()
    #开始训练
    # 图像增强队列
    #tf.train.start_queue_runners()

    for step in range(max_steps):
        start_time = time.time()
        images_batch,b_label = sess.run(train_next_batch)
        print("images_batch shape",b_image.shape)
        #获取一个批次的数据和标签
        #images_batch,labels_batch = sess.run([images_train,labels_train])
        b_image, b_label = sess.run([images_train, labels_train])
        _,loss_value,step_g = sess.run([train_step,loss,global_step],feed_dict={image_holder:b_image,
                                                                 label_holder:b_label})
        
        #获取计算时间
        duration = time.time() - start_time
        # 计算测试集准确率
        images_b,labels_b = sess.run([images_batch_test,labels_batch_test])
        #print("1",images_batch_test.shape)
        acc,loss_value_test = sess.run([accuracy,loss],feed_dict={image_holder:images_b,
                                                                 label_holder:labels_b}) 
        #计算每秒处理多少张图片
        per_images_second = batch_size / duration
        #获取时间
        sec_per_batch = float(duration)
        # 降低学习率
        lr_decay_val = sess.run(lr_decay) 
        sess.run(tf.assign(lr, lr_decay_val))     
        learning_rate = sess.run(lr)    
        sys.stdout.write('\r>> Step:%d,  duration:%.3f,  per_images_second:%.2f,  loss:%.3f,  accuracy:%.3f,  test loss:%.3f,  learning_rate:%.5f' % 
                         (step_g,duration,per_images_second,loss_value,acc,loss_value_test,learning_rate))
        sys.stdout.flush() 
        
        if step % 500 == 0:
            # 训练完成后保存训练模型
            saver.save(sess, "./captcha/models/crack_captcha.model", global_step=global_step, meta_graph_suffix='meta', write_meta_graph=True) 
            tf.train.write_graph(sess.graph_def, "./captcha/models", "crack_captcha.pb", False)
        

Instructions for updating:
Colocations handled automatically by placer.
Tensor("arg0:0", shape=(), dtype=string)
Tensor("arg0:0", shape=(), dtype=string)
